In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
# from selenium.webdriver.common.by import By

In [2]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# URL of page to be scraped
url = 'https://npin.cdc.gov/search/organization/testing/hiv'
#adding in splinter needed item
browser.visit(url)

#Scrape by state search function

In [5]:
# Retrieve page with the requests module
response = requests.get(url)
print(response.text)

<!DOCTYPE html>
<!--[if lt IE 7]><html class="lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->
<!--[if IE 7]><html class="lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="lt-ie9" lang="en" dir="ltr"><![endif]-->
<!--[if gt IE 8]><!--><html lang="en" dir="ltr" prefix="fb: http://www.facebook.com/2008/fbml content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# schema: http://schema.org/"><!--<![endif]-->
<head>
  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<meta charset="utf-8" />
<link rel="shortcut icon" href="https://npin.cdc.gov/sites/all/themes/custom/cdcnpin3/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="viewport" content="width=device-width, initia

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]><html class="lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->
<!--[if IE 7]><html class="lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->
<!--[if IE 8]><html class="lt-ie9" lang="en" dir="ltr"><![endif]-->
<!--[if gt IE 8]><!-->
<html dir="ltr" lang="en" prefix="fb: http://www.facebook.com/2008/fbml content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# schema: http://schema.org/">
 <!--<![endif]-->
 <head>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <link href="https://npin.cdc.gov/sites/all/themes/custom/cdcnpin3/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <meta content="width=device-width, initial-scale=1

In [7]:
# results are returned as an iterable list
results = soup.find_all('div', class_="ds-1col node node-organization article user-picture-available view-mode-search_result clearfix")
for r in results:
    print(r.prettify())

<div about="/organization/southwest-center-hivaids" class="ds-1col node node-organization article user-picture-available view-mode-search_result clearfix" role="article" typeof="sioc:Item foaf:Document">
 <div class="field field-name-title field-type-ds field-label-hidden view-mode-search_result">
  <div class="field-items">
   <div class="field-item even" property="dc:title">
    <h3>
     <a href="/organization/southwest-center-hivaids">
      Southwest Center for HIV/AIDS
     </a>
    </h3>
   </div>
  </div>
 </div>
 <div class="field field-name-serp-content-type field-type-ds field-label-hidden view-mode-search_result">
  <div class="field-items">
   <div class="field-item even">
    <span class="node-content-type">
     Organization
    </span>
   </div>
  </div>
 </div>
 <section class="field field-name-organization-inline field-type-ds field-label-inline clearfix view-mode-search_result">
  <h2 class="field-label">
   Location:
  </h2>
  <div class="field-items">
   <div class

In [8]:
organizations = []

#Iterate through all pages (two at first)
for x in range(889):
    #HTML object
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    results = soup.find_all('div', class_="ds-1col node node-organization article user-picture-available view-mode-search_result clearfix")
    
    #Iterate through each organization using Beautiful Soup's find methods
    for r in results:
        
        #returns organization name
        
        #returns organization name (first line)
        try:
            n_step = r.find(class_='field')
            n_step2 = n_step.find('a')
            name = n_step2.text
        except:
            name = ' '
        #returns organization name (second line)    
        try:
            n2_step = r.find(class_='field-name-field-organization-name-2')
            n2_step2 = n2_step.find('a')
            name2 = n2_step2.text
        except (NameError, AttributeError):
            name2 = ' '
        full_name = (name + ' ' + name2)
        
        #returns organization address
        try:
            a_step = r.find('section', class_='field')
            a_step2 = a_step.find(class_='field-item even')
            address = a_step2.text
        except:
            address = 'need info'
                    
        #returns organization phone number    
        try:
            p_step = r.find(class_='field-name-phone-number')
            p_step2 = p_step.find(class_='field-item even')
            phone_num = p_step2.text
        except:
            phone_num = 'need info'
            
#       print(name)
#       print(address)
#       print(phone_num)
        organizations.append({'Name':full_name, 'Address':address, 'Phone Number':phone_num})
        
    #Click the 'next' button on each page (will include try/except after I know it works)
#    time.sleep(5)  # wait 5 seconds before clicking next button
#    types of errors AttributeError, ElementDoesNotExist, IndexError
    try:
        browser.find_by_css('a.fsrDeclineButton').click()
    except:
        try:
            browser.find_by_css('li.pager-next a.action-item').click()
        except Exception as e:
            print(e)


In [9]:
org_df = pd.DataFrame(organizations, columns=['Name', 'Address', 'Phone Number'])
org_df

,Name,Address,Phone Number
0,Southwest Center for HIV/AIDS,"1101 N Central Ave, Ste 200, Phoenix, Arizona ...",(602) 307-5330
1,Arnot Health HIV Elmira Primary Care Clinic,"600 Roe Ave, Ste 1 F, Elmira, New York 14905, ...",(607) 795-8161
2,Provident Hospital HIV Clinic HIV Program,"500 E 51st St, Rm 7067, Chicago, Illinois 6061...",(312) 572-2728
3,South Suburban HIV/AIDS Regional Clinics,"15900 S Cicero Ave, Bldg B-11, Oak Forest, Ill...",(708) 633-2845
4,UNIFIED - HIV Health and Beyond,"3968 Mt Elliott St, Detroit, Michigan 48207, U...",(313) 446-9800
5,HIV/ HCV Resource Center,"2 Blacksmith St, Lebanon, New Hampshire 03766,...",(603) 448-8887
6,Delaware HIV Consortium,"100 W 10th St, Ste 415, Wilmington, Delaware 1...",(302) 654-5471
7,Allies Linked for the Prevention of HIV and AI...,"537 W Bannock St, Ste 100, Boise, Idaho 83702,...",(208) 424-7799
8,Southern New Hampshire HIV/AIDS Task Force,"77 Northeastern Blvd, Nashua, New Hampshire 03...",(603) 595-8464
9,Fredericksburg Area HIV/AIDS Support Services ...,"415 Elm St, Fredericksburg, Virginia 22401, Un...",(540) 371-7532


In [10]:
org_df['Address'] = org_df['Address'].str.replace(',', '')
org_df['Address']

0       1101 N Central Ave Ste 200 Phoenix Arizona 850...
1       600 Roe Ave Ste 1 F Elmira New York 14905 Unit...
2       500 E 51st St Rm 7067 Chicago Illinois 60615 U...
3       15900 S Cicero Ave Bldg B-11 Oak Forest Illino...
4       3968 Mt Elliott St Detroit Michigan 48207 Unit...
5       2 Blacksmith St Lebanon New Hampshire 03766 Un...
6       100 W 10th St Ste 415 Wilmington Delaware 1980...
7       537 W Bannock St Ste 100 Boise Idaho 83702 Uni...
8       77 Northeastern Blvd Nashua New Hampshire 0306...
9       415 Elm St Fredericksburg Virginia 22401 Unite...
10      1195 A City View St Eugene Oregon 97402 United...
11      5323 Foothill Blvd Oakland California 94601 Un...
12      Corner Of Navajo Rte 9 and Chaco Blvd Crownpoi...
13      647 W Luellen Ste 3 Roseburg Oregon 97470 Unit...
14      17 Dunbar St Keene New Hampshire 03431 United ...
15      74-5620 Palani Rd Ste 101 Kailua Kona Hawaii 9...
16      101 Aupuni St Penthouse 1014C Hilo Hawaii 9672...
17      P O Bo

In [12]:
# sorting by first name 
org_df.sort_values('Name', inplace = True) 
  
# making a bool series 
bool_series = org_df['Name'].duplicated() 
  
# displaying data 
org_df.head() 
  
# display data 
org_df[bool_series]

,Name,Address,Phone Number
2526,ACCESS Community Health,4301 E 14 Mile Rd Sterling Heights Michigan 48...,(586) 722-6036
2527,ACCESS Community Health,6450 Maple St Dearborn Michigan 48126 United S...,(313) 216-2200
782,ACCESS Community Health,963 Allen St Ferndale Michigan 48220 United St...,(248) 629-7629
2729,ACR Health,627 W Genesee St Syracuse New York 13204 Unite...,(315) 475-2430
4269,AFC Urgent Care,38 Boston Post Rd W Marlborough Massachusetts ...,(508) 658-0764
3847,AFC Urgent Care,3059 Solomons Island Rd Ste F-2 Edgewater Mary...,(410) 956-3394
7355,AFC Urgent Care,1030 Boulevard West Hartford Connecticut 06119...,(860) 986-6440
4294,AFC Urgent Care,376 Arsenal St Watertown Massachusetts 02472 U...,(617) 923-2273
356,AID Upstate,811 Pendleton St Stes 10 & 11 Greenville South...,(864) 250-0607
297,AIDS Care Group,2201 Providence Ave Chester Pennsylvania 19013...,(610) 872-9101


In [11]:
# Export file as a CSV, without the Pandas index, but with the header
org_df.to_csv("hiv_providers.csv", index=True, header=True)

In [ ]:
# # Imports the method used for connecting to DBs
# from sqlalchemy import create_engine
# # Imports the methods needed to abstract classes into tables
# from sqlalchemy.ext.declarative import declarative_base
# # Allow us to declare column types
# from sqlalchemy import Column, Integer, String, Float
# from sqlalchemy.orm import Session

In [ ]:
 # Create Organization Class
    
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Classes which will serve as the anchor points for our Tables
class Organization(Base):
    __tablename__ = 'org_data'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    address = Column(String(255))
    phone_num = Column(String(255))

In [ ]:
 # Create Engine for organization data
# org_db_path = "../hiv_providers.sqlite"
engine = create_engine(f"sqlite:///org_data.sqlite")
conn = engine.connect()

In [ ]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# Base.metadata.drop_all(engine)

In [ ]:
# Create a Session Object to Connect to DB
# Session is a temporary binding to our DB
session = Session(bind=engine)

In [ ]:
# Query All Records in the the Organizations Table
org_data = pd.read_sql("SELECT * FROM Org_Data", conn)
org_data

In [ ]:
org_df = pd.read_csv("hiv_providers.csv")
org_df.head()

In [ ]:
org_df['Address'] = org_df['Address'].str.replace(',', '')

In [ ]:
org_df['Address']

In [ ]:
# Display items in MongoDB collection
listings = db.items.find()

for listing in listings:
    print(listing)